# Basics

Let's first take a look at what's inside the ``ib_async`` package:

In [1]:
import ib_async
print(ib_async.__all__)

['util', 'Event', 'OrderComboLeg', 'OrderState', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'TagValue', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'ComboLeg', 'DeltaNeutralContract', 'Con

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [2]:
from ib_async import *
util.startLoop()

*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [3]:
ib = IB()
ib.connect('127.0.0.1', 4001, clientId=10)

<IB connected to 127.0.0.1:4001 clientId=10>

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [4]:
ib.positions()

[Position(account='U2340948', contract=Stock(conId=4762, symbol='BA', exchange='NYSE', currency='USD', localSymbol='BA', tradingClass='BA'), position=254.0, avgCost=203.21112205),
 Position(account='U2340948', contract=Option(conId=699423951, symbol='BA', lastTradeDateOrContractMonth='20240607', strike=190.0, right='C', multiplier='100', currency='USD', localSymbol='BA    240607C00190000', tradingClass='BA'), position=1.0, avgCost=92.04685),
 Position(account='U2340948', contract=Stock(conId=47486780, symbol='EDV', exchange='ARCA', currency='USD', localSymbol='EDV', tradingClass='EDV'), position=650.0, avgCost=77.7080769),
 Position(account='U2340948', contract=Stock(conId=229726228, symbol='RSX', exchange='VALUE', currency='USD', localSymbol='RSX', tradingClass='RSX'), position=100.0, avgCost=6.889977),
 Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSy

In [5]:
ib.accountValues()

[AccountValue(account='U10394496', tag='AccountCode', value='U10394496', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='AccountReady', value='true', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='ColumnPrio-C', value='2', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='ColumnPrio-P', value='5', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='ColumnPrio-S', value='1', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='Cushion', value='0.778792', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='DayTradesRemaining', value='-1', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='DayTradesRemainingT+1', value='-1', currency='', modelCode=''),
 AccountValue(account='U10394496', tag='DayTradesRemainingT+2', value='-1', currency='', modelCode=''),
 AccountValue

Or filter the account values to get the liquidation value:

In [5]:
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='U172554', tag='NetLiquidationByCurrency', value='2039.4021', currency='BASE', modelCode='')]

The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [6]:
Contract(conId=270639)
Stock('AMD', 'SMART', 'USD')
Stock('INTC', 'SMART', 'USD', primaryExchange='NASDAQ')
Forex('EURUSD')
CFD('IBUS30')
Future('ES', '20180921', 'GLOBEX')
Option('SPY', '20170721', 240, 'C', 'SMART')
Bond(secIdType='ISIN', secId='US03076KAA60');

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [7]:
contract = Stock('TSLA', 'SMART', 'USD')
ib.reqContractDetails(contract)

[ContractDetails(contract=Contract(secType='STK', conId=76792991, symbol='TSLA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TSLA', tradingClass='NMS', comboLegs=[]), marketName='NMS', minTick=0.01, orderTypes='ACTIVETIM,ADJUST,ALERT,ALGO,ALLOC,AON,AVGCOST,BASKET,BENCHPX,COND,CONDORDER,DARKONLY,DARKPOLL,DAY,DEACT,DEACTDIS,DEACTEOD,DIS,GAT,GTC,GTD,GTT,HID,IBKRATS,ICE,IMB,IOC,LIT,LMT,LOC,MIDPX,MIT,MKT,MOC,MTL,NGCOMB,NODARK,NONALGO,OCA,OPG,OPGREROUT,PEGBENCH,POSTONLY,PREOPGRTH,REL,RPI,RTH,SCALE,SCALEODD,SCALERST,SNAPMID,SNAPMKT,SNAPREL,STP,STPLMT,SWEEP,TRAIL,TRAILLIT,TRAILLMT,TRAILMIT,WHATIF', validExchanges='SMART,AMEX,NYSE,CBOE,PHLX,ISE,CHX,ARCA,ISLAND,DRCTEDGE,BEX,BATS,EDGEA,CSFBALGO,JEFFALGO,BYX,IEX,EDGX,FOXRIVER,TPLUS1,NYSENAT,PSX', priceMagnifier=1, underConId=0, longName='TESLA INC', contractMonth='', industry='Consumer, Cyclical', category='Auto Manufacturers', subcategory='Auto-Cars/Light Trucks', timeZoneId='EST (Eastern Standard Time)', tradingHours

### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [8]:
%time l = ib.positions()

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 9.06 µs


In [9]:
%time l = ib.reqPositions()

CPU times: user 0 ns, sys: 745 µs, total: 745 µs
Wall time: 32.7 ms


### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [10]:
util.logToConsole()

To see all debug messages (including network traffic):

In [11]:
import logging
util.logToConsole(logging.DEBUG)

### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [12]:
ib.disconnect()

2019-12-31 13:28:29,252 ib_async.ib INFO Disconnecting from 127.0.0.1:7497, 160 B sent in 9 messages, 21.0 kB received in 418 messages, session time 920 ms.
2019-12-31 13:28:29,255 ib_async.client INFO Disconnecting
